In [7]:
import pandas as pd
import re
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [9]:
train_limpio = pd.read_csv("train_limpio.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,...,blown,bridge,crush,derailment,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died,yeas
0,"['reason', 'earthquake', 'may', 'forgive', 'us']",1,0,1,70,1,6,0,0,1,...,0,0,0,0,2,0,0,0,0,0
1,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...",1,0,1,40,0,6,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
df_x=train_limpio.drop(['text', 'target'], axis=1)
df_x.head()

,cantidad_links,repeticiones,longitud_tweet,the,a,to,in,of,and,i,...,blown,bridge,crush,derailment,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died,yeas
0,0,1,70,1,6,0,0,1,0,2,...,0,0,0,0,2,0,0,0,0,0
1,0,1,40,0,6,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,1,130,1,7,1,3,1,0,8,...,0,0,0,0,2,0,0,0,0,0
3,0,1,60,0,4,0,1,0,0,7,...,0,0,0,0,1,0,0,0,0,0
4,0,1,90,0,4,2,1,0,0,4,...,0,0,0,0,2,0,0,0,0,0


In [11]:
df_y=train_limpio.target
df_y

0       1
1       1
2       1
3       1
4       1
       ..
7126    1
7127    1
7128    1
7129    1
7130    1
Name: target, Length: 7131, dtype: int64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [13]:
rf= RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
pred = rf.predict(x_test)
pred

array([1, 0, 0, ..., 0, 1, 0])

In [15]:
y_test.values

array([1, 0, 0, ..., 0, 1, 1])

In [16]:
count = 0
for i in range( len(pred)):
    if pred[i]==y_test.values[i]:
        count = count+1
print(count)
print(len(pred))
print(count/len(pred))

1069
1427
0.7491240364400841
